# Transcribe Recordings

paid service [Goodtape](http://goodtape.io) will be used for the transcriptions

## Imports

In [27]:
%reload_ext autoreload
%autoreload 2

In [28]:
import requests
import pandas as pd
import os
import docx

# local imports
from lib_henryk.logger import *
from lib_henryk.config import *
from lib_henryk.utils import *

from lib_henryk.recordings import transcription
from lib_henryk.recordings import recordings

## Params

In [29]:
# load api keys
from dotenv import load_dotenv
_ = load_dotenv()

# retrieve new token by opening 'https://webhook.site/' in incognito mode
# and picking up the token from the URL
WEBHOOKS_TOKEN_ID = '035fb8e4-5260-4e10-9e86-5782fb3bb352'
GOODTAPE_API_KEY = os.getenv("GOODTAPE_API_KEY")

## Load Dataset

In [30]:
# get fresh stats directly from the recordings
df = recordings.get_recordings_info(path_recordings=DIR_WIADOMOSCI_DO_HENRYCZKA)

2024-05-12 23:52:25 - INFO  - get_recordings_info - retrieved 503 files
processing 503 audio files |██████████████████████████████████████████████████| 100.0% 


## Run Transcriptions

In [31]:
# perform transcriptions
transcription.submit_transcriptions_goodtape(
    df=df, 
    webhooks_token_id=WEBHOOKS_TOKEN_ID,
    goodtape_api_key=GOODTAPE_API_KEY,
    path_recordings=DIR_WIADOMOSCI_DO_HENRYCZKA,
    start=0, stop=510,
    verbose=False
)

processing [78]: Henryk 2024-05-05 Tatuś opowiada o festynie a Kaudipteryks przechwala się ogonem.m4a
processing [79]: Henryk 2024-05-06 Tatuś opowiada o rekrutacji a Megaterium to największy ssak na ziemi.m4a
processing [80]: Henryk 2024-05-08 Tatuś opowiada o robieniu komputera do obliczeń a Ceratopsy gonią złodzieja jaj...
processing [81]: Henryk 2024-05-10 Tatuś opowiada o miejscach do których pójdziemy a Ichtiozaury zmieniają zwycza...
processing [85]: Rodzice Henryk 2024-05-05 Nagranie z wycieczki babci i dziadka.m4a
submitted 5 recordings to the transcription service GoodTape


## Fetch Transcriptions JSONs

In [7]:
# need to wait until transcriptions are available
exit_cell()

StopExecution: stopped

In [34]:
# retrieve goodtape server responses from webhooks
transcription.retrieve_responses_goodtape_via_webhooks(
    df=df,
    webhooks_token_id=WEBHOOKS_TOKEN_ID, 
    path_transcriptions_json=DIR_TRANSCRIPTIONS_JSON
)

# process json to text
transcription.process_transcriptions_json(
    df=df, 
    path_transcriptions_json=DIR_TRANSCRIPTIONS_JSON, 
    path_transcriptions_txt=DIR_TRANSCRIPTIONS_TXT,
    path_transcriptions_doc=DIR_TRANSCRIPTIONS_DOC,
    to_doc=True,
    to_txt=True,
    verbose=False,
)

# process json to doc

found 4 requests to fetch
retrieved transcription [0]: abef8eb5-1c43-461c-ac09-10dc35ac691b -> file [81]: Henryk 2024-05-10 Tatuś opowiada o miejscach do których pójdz...
retrieved transcription [1]: b575370a-df1c-43e5-8922-0ca9dad3efa4 -> file [85]: Rodzice Henryk 2024-05-05 Nagranie z wycieczki babci i dziadka.m...
retrieved transcription [2]: 2ff72b9b-9f61-44ba-883b-192303965060 -> file [79]: Henryk 2024-05-06 Tatuś opowiada o rekrutacji a Megaterium to n...
retrieved transcription [3]: b799e08c-b412-4fa2-a204-faac598c36f7 -> file [80]: Henryk 2024-05-08 Tatuś opowiada o robieniu komputera do oblicz...
4 transcriptions were retrieved
processing 503 transcriptions |██████████████████████████████████████████████████| 100.0% done.
4 new transcriptions were processed, there are 503 txt and 503 doc transcriptions available


## Cleanup missing requests
- transcription ids for the requests that were lost will be removed, only those that were processed will remain

In [31]:
# cleanup stale transcription ids - but work on a copy of the log
df_transcriptions_to_write = transcription.get_verified_transcriptions(
    df=df, 
    path_transcriptions_json=DIR_TRANSCRIPTIONS_JSON, 
    path_transcriptions_txt=DIR_TRANSCRIPTIONS_TXT,
    path_transcriptions_doc=DIR_TRANSCRIPTIONS_DOC,
)

# save transcription log
df_transcriptions_to_write.to_csv(FILE_TRANSCRIPTION_CSV, index=False)

verifying 498 transcriptions |██████████████████████████████████████████████████| 100.0% done.
there are 498 processed and valid transcriptions
*** all 498 recordings were transcribed ***
